# Getting started with the Python environment and packages

In [1]:
# import the relevant Python packages

import numpy
import nibabel
import nipype
import matplotlib


# The PET Brain Imaging Data Structure

## History

The PET modality is a recent addition to BIDS with its introduction via BEP 009. If you're interested in seeing exactly what and how something gets added to BIDS see the pull request for BEP009 [here](https://github.com/bids-standard/bids-specification/pull/633). The results of that extension proposal can be read [here](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/09-positron-emission-tomography.html#positron-emission-tomography) in the bids standard.

# PET data conversion

## PET image data file formats


Before we start to convert data we need to quickly mention that PET image data files come of the scanner in various different formats, some scanners provide DICOM files (.dcm) and others use proprietary formats for example ECAT format (.v) . In order to facilitate easy testing of data conversion across different PET file formats  the [OpenNeuroPET project](https://openneuropet.github.io/) has compiled a bunch of phantom data from different scanner types and is distributing two examples [here](https://drive.google.com/file/d/10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD/view?usp=sharing) . You can download them for testing purposes either manually or in the terminal by typing 

In [2]:
!pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=7b5da9f5dcbf45101c73750f5f7e7b7649ae36c72b5a3c2d7baff428d8cbc711
  Stored in directory: /home/jovyan/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown


In [3]:
!gdown https://drive.google.com/file/d/10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD/view?usp=sharing --fuzzy

Downloading...
From: https://drive.google.com/uc?id=10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD
To: /home/jovyan/OpenNeuroPET-Demo_raw.zip
100%|██████████████████████████████████████| 53.7M/53.7M [00:06<00:00, 7.68MB/s]


Now let's look at wat we have downloaded:

In [4]:
!ls .

apt.txt		 images   OpenNeuroPET-Demo_raw.zip  README.md
environment.yml  LICENSE  PET_BIDS_tutorial.ipynb    Untitled.ipynb


The correct file is there, so let's unzip the downloaded file:

In [5]:
!unzip OpenNeuroPET-Demo_raw.zip

Archive:  OpenNeuroPET-Demo_raw.zip
   creating: OpenNeuroPET-Demo_raw/
  inflating: __MACOSX/._OpenNeuroPET-Demo_raw  
  inflating: OpenNeuroPET-Demo_raw/.DS_Store  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._.DS_Store  
   creating: OpenNeuroPET-Demo_raw/sub-SiemensHRRT/
   creating: OpenNeuroPET-Demo_raw/source/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._source  
   creating: OpenNeuroPET-Demo_raw/code/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._code  
  inflating: OpenNeuroPET-Demo_raw/README  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._README  
  inflating: OpenNeuroPET-Demo_raw/.bidsignore  
  inflating: OpenNeuroPET-Demo_raw/dataset_description.json  
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._dataset_description.json  
   creating: OpenNeuroPET-Demo_raw/sub-SiemensBiographNRU/
  inflating: __MACOSX/OpenNeuroPET-Demo_raw/._sub-SiemensBiographNRU  
   creating: OpenNeuroPET-Demo_raw/sub-SiemensHRRT/pet/
   creating: OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR

You can now look at the file tree:

In [6]:
!tree OpenNeuroPET-Demo_raw

OpenNeuroPET-Demo_raw
├── code
│   ├── matlab_conversions.m
│   ├── python_conversions.sh
│   └── README.md
├── dataset_description.json
├── README
├── source
│   ├── SiemensBiographPETMR-NRU
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.100.2022.04.26.15.04.22.218.14690618.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.101.2022.04.26.15.04.22.218.14690629.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.10.2022.04.26.15.04.22.218.14689628.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.102.2022.04.26.15.04.22.218.14690640.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.103.2022.04.26.15.04.22.218.14690651.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.104.2022.04.26.15.04.22.218.14690662.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.105.2022.04.26.15.04.22.218.14690673.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.106.2022.04.26.15.04.22.218.14690684.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_xca.30003.107.2022.04.26.15.04.22.218.14690695.dcm
│   │   ├── X-CAL_7.PT.Kalibrering_

Now you have an example dataset where you have source data (both for ECAT and DICOM PET image format) and the PET BIDS data sets constructed for it.

Also if you have access to another PET image file format, or data from a scanner not tested, please reach out to [OpenNeuroPET project](https://openneuropet.github.io/) in order to add a phantom scan in your format.

## Conversion

The [OpenNeuroPET project](https://openneuropet.github.io/) has tried to develop tools for facilitating easy data conversion for PET. The main tool used for this is [PET2BIDS](https://github.com/openneuropet/PET2BIDS) freely available on the [OpenNeuroPET  GitHub repository](https://github.com/openneuropet) along with other resources like altlases or pipelines. It is available for both Python and MatLab. Eventually, [PET2BIDS](https://github.com/openneuropet/PET2BIDS) will also be wrapped inside other BIDS conversion tools such as [BIDScoin](https://github.com/Donders-Institute/bidscoin) or [ezBIDS](https://brainlife.io/ezbids/), but this is work in progress at the moment. 

Besides using  [PET2BIDS](https://github.com/openneuropet/PET2BIDS) there is always the possibility to manually convert a data set to PET BIDS and an example will be shown below. In any case, for dicom data format, one relies on [dcm2niix](https://www.nitrc.org/plugins/mwiki/index.php/dcm2nii:MainPage).

Below we will show two ways of converting your PET data to BIDS: 1) using [PET2BIDS](https://github.com/openneuropet/PET2BIDS) and 2) manually.

### 1) Conversion of PET data using PET2BIDS

Detailed documentation for PET2BIDS can be found [here](https://pet2bids.readthedocs.io/en/latest/index.html#) or on the [Github repo](https://github.com/openneuropet/PET2BIDS/blob/main/README.md).



#### Get the Python package PET2BIDS

In [7]:
pip install pypet2bids 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.7/280.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 8.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.2/555.2 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.5 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### Convert your first dataset 

Now you already have the converter installed and can go ahead and convert your first dataset! 

In this example, I am converting an image in DICOM format. It should be noted that pypet2bids  contains several different tools and is itself a part of the larger PET library PET2BIDS, the specific tool I will be using for the following DICOM conversion is dcm2niix4pet.

You just need to point dcm2niix4pet to the folder where your data resides, *dcmfolder*, and the folder where you want to output the PET BIDS formatted dataset, *mynewfolder*:

In [8]:
!dcm2niix4pet ./OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR-NRU -d mynewfolder

Attempting to locate missing BIDS fields in dicom header
FOUND BodyPartExamined corresponding to BIDS BodyPart: BRAIN
NOT FOUND MappingResourceName corresponding to BIDS TracerName in dicom header.
NOT FOUND CodeMeaning corresponding to BIDS TracerRadionuclide in dicom header.
NOT FOUND RadionuclideTotalDose corresponding to BIDS InjectedRadioactivity in dicom header.
NOT FOUND RadiopharmaceuticalSpecificActivity corresponding to BIDS MolarActivity in dicom header.
NOT FOUND RadiopharmaceuticalVolume corresponding to BIDS InjectedVolume in dicom header.
NOT FOUND InterventionDrugName corresponding to BIDS PharmaceuticalName in dicom header.
NOT FOUND InterventionDrugDose corresponding to BIDS PharmaceuticalDoseAmount in dicom header.
NOT FOUND RadiopharmaceuticalStartTime corresponding to BIDS InjectionStart in dicom header.
FOUND AcquisitionDate corresponding to BIDS ScanDate: 20220421
NOT FOUND RadiopharmaceuticalStopTime corresponding to BIDS InjectionEnd in dicom header.
FOUND Reco

Above you can see some complaints that the converter has - NOT FOUND or FOUND. Note, dcm2niix4pet will do it's best to extract as much information about radiological and blood data from the DICOM files in the dcmfolder. However, dcm2niix4pet can't find information if it isn't there, hence it will often be up to you the user to provide some missing information at the time of conversion. 

Now we can also look at our BIDS dataset:

In [14]:
!tree ./mynewfolder

./mynewfolder
├── Phantom_PetAcquisition_20.01.14-10_39_18-STD-1.3.12.2.1107.5.2.38.51014_20220421120447_30003.json
└── Phantom_PetAcquisition_20.01.14-10_39_18-STD-1.3.12.2.1107.5.2.38.51014_20220421120447_30003.nii.gz

0 directories, 2 files


Additional information can be provided via the command line with the `--kwargs` argument in the form of key=pair values. For an idea of what this looks like see below:

In [16]:
!dcm2niix4pet ./OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR-NRU -d mynewfolder2 --kwargs TimeZero=ScanStart Manufacturer=Siemens ManufacturersModelName=Biograph InstitutionName="Rigshospitalet, NRU, DK" BodyPart=Phantom Units=Bq/mL TracerName=none TracerRadionuclide=F18 InjectedRadioactivity=81.24 SpecificRadioactivity=13019.23 ModeOfAdministration=infusion FrameTimesStart=0 AcquisitionMode="list mode" ImageDecayCorrected=true ImageDecayCorrectionTime=0 AttenuationCorrection=MR-corrected FrameDuration=300 FrameTimesStart=0

Attempting to locate missing BIDS fields in dicom header
FOUND BodyPartExamined corresponding to BIDS BodyPart: BRAIN
NOT FOUND MappingResourceName corresponding to BIDS TracerName in dicom header.
NOT FOUND CodeMeaning corresponding to BIDS TracerRadionuclide in dicom header.
NOT FOUND RadionuclideTotalDose corresponding to BIDS InjectedRadioactivity in dicom header.
NOT FOUND RadiopharmaceuticalSpecificActivity corresponding to BIDS MolarActivity in dicom header.
NOT FOUND RadiopharmaceuticalVolume corresponding to BIDS InjectedVolume in dicom header.
NOT FOUND InterventionDrugName corresponding to BIDS PharmaceuticalName in dicom header.
NOT FOUND InterventionDrugDose corresponding to BIDS PharmaceuticalDoseAmount in dicom header.
NOT FOUND RadiopharmaceuticalStartTime corresponding to BIDS InjectionStart in dicom header.
FOUND AcquisitionDate corresponding to BIDS ScanDate: 20220421
NOT FOUND RadiopharmaceuticalStopTime corresponding to BIDS InjectionEnd in dicom header.
FOUND Reco

Now you have a dataset in PET BIDS format. You will probably have gotten some warnings relating to the .json sidecar file. Carefully look at them, since they will help you to catch inconsistencies and missing required fields that you need to add in order for the dataset to pass the BIDS validator as well (see below how that's done). 
You can always edit the .json file, by opening it in a text editor and manually fixing errors. Alternatively, adjust the meta structure you created above to correct the errors.

# PET processing

## PET example data for processing

In the following we will basically follow the tutorial presented in the [PyPetSurfer repository](https://github.com/openneuropet/PET_pipelines/tree/main/pyPetSurfer) and perform a simple data processing of a PET data set. Therefore we need to clone that GitHub repository:


In [18]:
!mkdir PETprocessing

In [43]:
cd PETprocessing

[Errno 2] No such file or directory: 'PETprocessing'
/home/jovyan/PETprocessing


In [22]:
! git clone https://github.com/openneuropet/PET_pipelines.git .

Cloning into '.'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 54 (delta 12), reused 22 (delta 0), pack-reused 0
Receiving objects: 100% (54/54), 30.45 KiB | 2.54 MiB/s, done.
Resolving deltas: 100% (12/12), done.


Then we need to cd to the right directory inside the Github repository:

In [23]:
 !cd pyPetSurfer 

Download this example dataset from OpenNeuro: https://openneuro.org/datasets/ds001421. You can see instructions on how to do this [here](https://openneuro.org/datasets/ds001421/versions/1.2.1/download). In this example, I will use the openneuro cli 

In [25]:
!datalad install https://github.com/OpenNeuroDatasets/ds001421.git

It is highly recommended to configure Git before using DataLad. Set both 'user.name' and 'user.email' configuration variables.
Clone attempt:   0%|              | 0.00/2.00 [00:00<?, ? Candidate locations/s]
Enumerating: 0.00 Objects [00:00, ? Objects/s]
                                              
Counting:   0%|                              | 0.00/3.65k [00:00<?, ? Objects/s]
                                                                                
Compressing:   0%|                           | 0.00/2.62k [00:00<?, ? Objects/s]
                                                                                
Receiving:   0%|                             | 0.00/3.99k [00:00<?, ? Objects/s]
                                                                                
Resolving:   0%|                                | 0.00/381 [00:00<?, ? Deltas/s]
[INFO   ] Remote origin not usable by git-annex; setting annex-ignore           
[INFO   ] https://github.com/OpenNeuroDatasets/ds0

Now let's see if this is there and how it looks like:

In [45]:
!ls /home/jovyan/PETprocessing/pyPetSurfer/

ds001421-download  example.py  LICENSE	pet_surfer.py  __pycache__  README.md


As an idiosyncracy described in the [PyPetSurfer repository](https://github.com/openneuropet/PET_pipelines/tree/main/pyPetSurfer), we need to rename our folder where the dataset was downloaded to:

In [28]:
!mv ds001421 ./pyPetSurfer/ds001421-download

And check again that this is right:

In [32]:
!mv ds001421-download ./pyPetSurfer/ds001421-download

In [40]:
!cd ./pyPetSurfer/

And now simply perform your first PET data anlysis by running the example.py

In [46]:
%run ./pyPetSurfer/example.py


Directory ds001421-download does not exist. Attempting to dowload it using openneuro-cli (may take a few minutes)

openneuro download --snapshot 1.2.1 ds001421 ds001421-download
/bin/sh: 1: openneuro: not found



ValueError: The example dataset is not available. Please download it manually from https://openneuro.org/datasets/ds001421